# Calculate NxN kernel pixel stats on a SIR envi cube file

Do this for avg, std, min and max.
Currently uses the companion GRD .hdr file, but should really write its own headers.

In [1]:
%pylab notebook
import cetbtools
import matplotlib.pyplot as plt
import os
from osgeo import gdal, gdalconst
from osgeo.gdalconst import * 
import re
from shutil import copyfile

Populating the interactive namespace from numpy and matplotlib


In [13]:
%cd /scratch/summit/brodzik/tmp
%ls

/gpfs/summit/scratch/brodzik/tmp
CETB.cubefile.UIB.F13_SSMI-37V-GRD-CSU-v1.3.2009.TB.bin.hdr
CETB.cubefile.UIB.F13_SSMI-37V-SIR-CSU-v1.3.2009.TB.bin
CETB.cubefile.UIB.F13_SSMI-37V-SIR-CSU-v1.3.2009.TB.bin.hdr
CETB.cubefile.WesternUS.AQUA_AMSRE-18V-GRD-RSS-v1.3.2007.TB.bin
CETB.cubefile.WesternUS.AQUA_AMSRE-18V-GRD-RSS-v1.3.2007.TB.bin.hdr
CETB.cubefile.WesternUS.AQUA_AMSRE-18V-SIR-RSS-v1.3.2007.TB.bin
CETB.cubefile.WesternUS.AQUA_AMSRE-18V-SIR-RSS-v1.3.2007.TB.bin.hdr
CETB.cubefile.WesternUS.AQUA_AMSRE-36V-GRD-RSS-v1.3.2007.TB.bin
CETB.cubefile.WesternUS.AQUA_AMSRE-36V-GRD-RSS-v1.3.2007.TB.bin.hdr
CETB.cubefile.WesternUS.AQUA_AMSRE-36V-SIR-RSS-v1.3.2007.TB.bin
CETB.cubefile.WesternUS.AQUA_AMSRE-36V-SIR-RSS-v1.3.2007.TB.bin.hdr
CETB.cubefile.WesternUS.F13_SSMI-19V-GRD-CSU-v1.3.2007.TB.bin
CETB.cubefile.WesternUS.F13_SSMI-19V-GRD-CSU-v1.3.2007.TB.bin.hdr
CETB.cubefile.WesternUS.F13_SSMI-19V-SIR-CSU-v1.3.2007.TB.bin
CETB.cubefile.WesternUS.F13_SSMI-19V-SIR-CSU-v1.3.2007.TB.bin.hdr
CETB.cu

In [3]:
# Thanks to http://chris35wills.github.io/python-gdal-raster-io/
# for inspiration here
# The function can be called as follows:
# inDs, geotransform = openEnviCube(file_name)
def openEnviCube(file_name):
    '''
    Reads the ENVI cube and returns an object with its data and information
    Lack of an ENVI .hdr file will cause this to crash.
    '''

    inDs = gdal.Open(file_name, GA_ReadOnly)
    
    if inDs is None:
        raise IOError(
            "Could not open file=%s, possibly missing ENVI .hdr file?" % file_name)
    else:
        print("%s opened successfully" % file_name)
        print('~~~~~~~~~~~~~~')
        print('Get image size')
        print('~~~~~~~~~~~~~~')
        cols = inDs.RasterXSize
        rows = inDs.RasterYSize
        bands = inDs.RasterCount

        print("columns: %i" %cols)
        print("rows: %i" %rows)
        print("bands: %i" %bands)

        print('~~~~~~~~~~~~~~')
        print('Get georeference information')
        print('~~~~~~~~~~~~~~')
        geotransform = inDs.GetGeoTransform()
        originX = geotransform[0]
        originY = geotransform[3]
        pixelWidth = geotransform[1]
        pixelHeight = geotransform[5]

        print("origin x: %i" %originX)
        print("origin y: %i" %originY)
        print("width: %2.2f" %pixelWidth)
        print("height: %2.2f" %pixelHeight)

        return inDs, geotransform

In [14]:
#file = './CETB.cubefile.WesternUS.F13_SSMI-37V-SIR-CSU-v1.3.2007.TB.bin'
file = './CETB.cubefile.WesternUS.AQUA_AMSRE-18V-SIR-RSS-v1.3.2007.TB.bin'

In [15]:
inDs, geotransform = openEnviCube(file)

./CETB.cubefile.WesternUS.AQUA_AMSRE-18V-SIR-RSS-v1.3.2007.TB.bin opened successfully
~~~~~~~~~~~~~~
Get image size
~~~~~~~~~~~~~~
columns: 404
rows: 328
bands: 730
~~~~~~~~~~~~~~
Get georeference information
~~~~~~~~~~~~~~
origin x: 3075001
origin y: -6075000
width: 3075000.00
height: -6250.00


In [ ]:
#data, post, geotransform, inDs = ENVI_raster_binary_to_2d_array(file, 2)

In [ ]:
band = inDs.GetRasterBand(2)
data = band.ReadAsArray(0, 0, 280, 360)

In [ ]:
np.nanmean(data), np.nanstd(data), np.nanmin(data), np.nanmax(data)

In [5]:
# Calculate blocked stats by ksize X ksize blocks
# Will ignore values of zero or 60000
def getLayerStatsByKernel(data, ksize):
    
    #print("Data min/max = %.2f/%.2f" % (np.min(data), np.max(data)))
    
    rows, cols = data.shape
    
    # Confirm exact block size division
    if ((0 != mod(rows, ksize)) or (0 != mod(cols, ksize))):
        raise ValueError("Data shape must be divisible by ksize.")

    # Allocate arrays for stats
    new_rows = int(rows / ksize)
    new_cols = int(cols / ksize)
    outAvg = np.zeros((new_rows, new_cols))
    outStd = np.zeros((new_rows, new_cols))
    outMin = np.zeros((new_rows, new_cols))
    outMax = np.zeros((new_rows, new_cols))
    
    # Make a copy, and 
    # Set values of 0 or 60000 to nan so they get ignored
    not_set = data == 0
    missing = data == 60000
    tmp = data.astype(np.float32)
    tmp[not_set] = np.nan
    tmp[missing] = np.nan
    
    # print("Temp min/max = %.2f/%.2f" % (np.nanmin(tmp), np.nanmax(tmp)))
    
    # Step through the data in ksize X ksize blocks
    for x in np.arange(0, cols, ksize):
        for y in np.arange(0, rows, ksize):
            #print("\nNext block: (x, y)=(%d, %d)" % (x, y))
            view = tmp[y:y+ksize, x:x+ksize]
            
            new_row = int(y / ksize)
            new_col = int(x / ksize)
            outAvg[int(y / ksize), int(x / ksize)] = np.nanmean(view)
            outStd[int(y / ksize), int(x / ksize)] = np.nanstd(view)
            outMin[int(y / ksize), int(x / ksize)] = np.nanmin(view)
            outMax[int(y / ksize), int(x / ksize)] = np.nanmax(view)
            
    return {'avg':outAvg, 'std':outStd, 'min':outMin, 'max':outMax}


In [6]:
#out = getLayerStatsByKernel(data,8)

In [7]:
def prepOutputDir(file):
    
    # Make an output directory one level down from working file
    # Ignore errors if it already exists
    outDir = "%s/%s" % (os.path.dirname(file), 'SIRstats')
    try:                                                                                                                  
        os.makedirs(outDir, exist_ok=True)                                                                                                  
    except:
        raise                                                                                                         

    return outDir

In [ ]:
#outDir = prepOutputDir("./%s" % file)
#outDir

In [8]:
def copyCompanionGRDHdrFile(file, outDir):
    
    # Make sure there's a companion GRD cubefile hdr that will be used for output
    # envi cubes
    # This is a workaround for now, really I should just write the 25 km header
    src = re.sub(r'-SIR-', '-GRD-', file)
    src = re.sub(r'\.bin', '.bin.hdr', src)
    
    for ext in ['TBavg25km.bin.hdr', 'TBstd25km.bin.hdr',
                'TBmin25km.bin.hdr', 'TBmax25km.bin.hdr']:
        dst = "%s/%s" % (outDir, os.path.basename(file))
        dst = re.sub(r'TB\.bin', ext, dst)
        try:
            copyfile(src, dst)
        except:
            print("Error with companion GRD envi header %s" % outHdrFile)
            raise
        print("Made new envi hdr: %s to %s" % (src, dst))
    

In [9]:
file

'./CETB.cubefile.WesternUS.AQUA_AMSRE-36V-SIR-RSS-v1.3.2007.TB.bin'

In [ ]:
outDir = prepOutputDir(file)
copyCompanionGRDHdrFile(file, outDir)

In [10]:
def saveCubeData(origFilename, outDir, dataCube, statType):
    # statType : str, one of 'avg', 'std', 'min', 'max'
    # Save the stats cubes to flat binary files of np.uint16
    # which is ENVI data type 12
    dataCube = (dataCube + 0.5).astype(np.uint16)
    print("%s cube type is %s" % (statType, dataCube.dtype))
    print("%s cube min/max = %d, %d" % (statType, 
                                        np.min(dataCube[dataCube > 0]), 
                                        np.max(dataCube[dataCube > 0])))
    print("%s cube dims = %s" % (statType, str(dataCube.shape)))
    outfile = os.path.basename(origFilename)
    outfile = re.sub(r'TB\.bin', 'TB%s25km.bin' % statType, outfile)
    outfile = "%s/%s" % (outDir, outfile)
    dataCube.tofile(outfile)
    print("Wrote %sCube to %s" % (statType, outfile))

In [11]:
def getCubeStats(file):
    
    print("File: %s" % file)
    
    # Open the cubefile
    inDs, geotransform = openEnviCube(file)
    
    # Prep the output directory
    outDir = prepOutputDir(file)
    
    # Make 2 companion GRD Hdr files
    copyCompanionGRDHdrFile(file, outDir)
    
    # stats cubes kernels will depend on scale
    ksize = int(-1 * (25000. / geotransform[5]))
    print("Kernel will be: %d x %d pixels" % (ksize, ksize))
    
    # Allocate memory for stats cubes
    cols = inDs.RasterXSize
    rows = inDs.RasterYSize
    bands = inDs.RasterCount
    rows25km = int(rows / ksize)
    cols25km = int(cols / ksize)
    avgCube = np.zeros((bands, rows25km, cols25km))
    stdCube = np.zeros((bands, rows25km, cols25km))
    minCube = np.zeros((bands, rows25km, cols25km))
    maxCube = np.zeros((bands, rows25km, cols25km))
    
    # Loop through each time slice and calculate stats
    for bandnum in np.arange(inDs.RasterCount):
    #for bandnum in np.arange(2):
        if 0 == mod(bandnum, 100):
            print("band = %d" % bandnum)
        band = inDs.GetRasterBand(int(bandnum + 1))
        data = band.ReadAsArray(0, 0, cols, rows)
        out = getLayerStatsByKernel(data, ksize)
        avgCube[bandnum, :, :] = out['avg'].copy()
        stdCube[bandnum, :, :] = out['std'].copy()
        minCube[bandnum, :, :] = out['min'].copy()
        maxCube[bandnum, :, :] = out['max'].copy()

    saveCubeData(file, outDir, avgCube, 'avg')
    saveCubeData(file, outDir, stdCube, 'std')
    saveCubeData(file, outDir, minCube, 'min')
    saveCubeData(file, outDir, maxCube, 'max')

In [ ]:
%pwd
%ls

In [ ]:
file

In [16]:
getCubeStats(file)

File: ./CETB.cubefile.WesternUS.AQUA_AMSRE-18V-SIR-RSS-v1.3.2007.TB.bin
./CETB.cubefile.WesternUS.AQUA_AMSRE-18V-SIR-RSS-v1.3.2007.TB.bin opened successfully
~~~~~~~~~~~~~~
Get image size
~~~~~~~~~~~~~~
columns: 404
rows: 328
bands: 730
~~~~~~~~~~~~~~
Get georeference information
~~~~~~~~~~~~~~
origin x: 3075001
origin y: -6075000
width: 3075000.00
height: -6250.00
Made new envi hdr: ./CETB.cubefile.WesternUS.AQUA_AMSRE-18V-GRD-RSS-v1.3.2007.TB.bin.hdr to ./SIRstats/CETB.cubefile.WesternUS.AQUA_AMSRE-18V-SIR-RSS-v1.3.2007.TBavg25km.bin.hdr
Made new envi hdr: ./CETB.cubefile.WesternUS.AQUA_AMSRE-18V-GRD-RSS-v1.3.2007.TB.bin.hdr to ./SIRstats/CETB.cubefile.WesternUS.AQUA_AMSRE-18V-SIR-RSS-v1.3.2007.TBstd25km.bin.hdr
Made new envi hdr: ./CETB.cubefile.WesternUS.AQUA_AMSRE-18V-GRD-RSS-v1.3.2007.TB.bin.hdr to ./SIRstats/CETB.cubefile.WesternUS.AQUA_AMSRE-18V-SIR-RSS-v1.3.2007.TBmin25km.bin.hdr
Made new envi hdr: ./CETB.cubefile.WesternUS.AQUA_AMSRE-18V-GRD-RSS-v1.3.2007.TB.bin.hdr to ./SIRs

/projects/brodzik/miniconda3/envs/cetb/lib/python3.6/site-packages/ipykernel_launcher.py:39: RuntimeWarning: Mean of empty slice
/projects/brodzik/miniconda3/envs/cetb/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1434: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)
/projects/brodzik/miniconda3/envs/cetb/lib/python3.6/site-packages/ipykernel_launcher.py:41: RuntimeWarning: All-NaN slice encountered
/projects/brodzik/miniconda3/envs/cetb/lib/python3.6/site-packages/ipykernel_launcher.py:42: RuntimeWarning: All-NaN slice encountered


band = 100
band = 200
band = 300
band = 400
band = 500
band = 600
band = 700
avg cube type is uint16
avg cube min/max = 15400, 33876
avg cube dims = (730, 82, 101)
Wrote avgCube to ./SIRstats/CETB.cubefile.WesternUS.AQUA_AMSRE-18V-SIR-RSS-v1.3.2007.TBavg25km.bin
std cube type is uint16
std cube min/max = 1, 6580
std cube dims = (730, 82, 101)
Wrote stdCube to ./SIRstats/CETB.cubefile.WesternUS.AQUA_AMSRE-18V-SIR-RSS-v1.3.2007.TBstd25km.bin
min cube type is uint16
min cube min/max = 13958, 33876
min cube dims = (730, 82, 101)
Wrote minCube to ./SIRstats/CETB.cubefile.WesternUS.AQUA_AMSRE-18V-SIR-RSS-v1.3.2007.TBmin25km.bin
max cube type is uint16
max cube min/max = 15400, 34986
max cube dims = (730, 82, 101)
Wrote maxCube to ./SIRstats/CETB.cubefile.WesternUS.AQUA_AMSRE-18V-SIR-RSS-v1.3.2007.TBmax25km.bin


In [ ]:
%ls -las

In [ ]:
%pwd
%ls -las

In [ ]:
origFile = file
avgFile = "./SIRstats/CETB.cubefile.WesternUS.F13_SSMI-19V-SIR-CSU-v1.3.2007.TBavg25km.bin"
stdFile = "./SIRstats/CETB.cubefile.WesternUS.F13_SSMI-19V-SIR-CSU-v1.3.2007.TBstd25km.bin"
minFile = "./SIRstats/CETB.cubefile.WesternUS.F13_SSMI-19V-SIR-CSU-v1.3.2007.TBmin25km.bin"
maxFile = "./SIRstats/CETB.cubefile.WesternUS.F13_SSMI-19V-SIR-CSU-v1.3.2007.TBmax25km.bin"

In [ ]:
origDs, origGeotransform = openEnviCube(origFile)
avgDs, avgGeotransform = openEnviCube(avgFile)
stdDs, stdGeotransform = openEnviCube(stdFile)
minDs, minGeotransform = openEnviCube(minFile)
maxDs, maxGeotransform = openEnviCube(maxFile)


In [ ]:
# UIB, 3->25
#origrows = 280
#origcols = 360
#outrows = 45
#outcols = 35
# WesternUS 3->25
#origrows = 656
#origcols = 808
# WesternUS 6->25
origrows = 326
origcols = 404
outrows = 82
outcols = 101
bandnum = 101
origBand = origDs.GetRasterBand(bandnum)
origData = origBand.ReadAsArray(0, 0, origcols, origrows)
avgBand = avgDs.GetRasterBand(bandnum)
avgData = avgBand.ReadAsArray(0, 0, outcols, outrows)
stdBand = stdDs.GetRasterBand(bandnum)
stdData = stdBand.ReadAsArray(0, 0, outcols, outrows)
minBand = minDs.GetRasterBand(bandnum)
minData = minBand.ReadAsArray(0, 0, outcols, outrows)
maxBand = maxDs.GetRasterBand(bandnum)
maxData = maxBand.ReadAsArray(0, 0, outcols, outrows)

In [ ]:
ksize = 4
fig, ax = plt.subplots(2,3, figsize=(8,6))
ax[0,0].imshow(origData/100., cmap=plt.cm.gray, interpolation='None', vmin=np.min(50.), vmax=np.max(300.))
ax[0,0].set_title("Orig (tidx=%d)" % (bandnum-1))
ax[0,1].imshow(avgData/100., cmap=plt.cm.gray, interpolation='None', vmin=np.min(50.), vmax=np.max(300.))
ax[0,1].set_title("%dx%d Avg" % (ksize, ksize))
ax[0,2].imshow(stdData/100., cmap=plt.cm.gray, interpolation='None', vmin=np.min(0.), vmax=np.max(5.))
ax[0,2].set_title("%dx%d Std" % (ksize, ksize))
ax[1,1].imshow(minData/100., cmap=plt.cm.gray, interpolation='None', vmin=np.min(50.), vmax=np.max(300.))
ax[1,1].set_title("%dx%d Min" % (ksize, ksize))
ax[1,2].imshow(maxData/100., cmap=plt.cm.gray, interpolation='None', vmin=np.min(50.), vmax=np.max(300.))
ax[1,2].set_title("%dx%d Max" % (ksize, ksize))
plt.tight_layout()
fig.savefig("./test.SIR-avg-std-min-max.t%03d.png" % (bandnum-1), dpi=300)



In [ ]:
file


In [ ]:
testfile = '/this/is/a/file.nc'
os.path.basename(testfile)

In [ ]:
outfile = re.sub("", "abcdef")

In [ ]:
# Figure out how to write out the cube with appropriate .hdr file 
# Or just copy the 25 km header?



In [ ]:
data.shape

In [ ]:
# Thanks to http://chris35wills.github.io/python-gdal-raster-io/
# for inspiration here
# The function can be called as follows:
# image_array, post, envidata =  ENVI_raster_binary_to_2d_array(file_name, band_num) 
#
# band_num = 0 for first time slice in the file
# Notes:
# Notice the tuple (geotransform, inDs) - this contains all of your map information 
# (xy tie points, postings and coordinate system information)
# pixelWidth is assumed to be the same as pixelHeight in the above example, therefore 
# representing the surface posting - if this is not the case for your data then you 
# must change the returns to suit
def ENVI_raster_binary_to_2d_array(file_name, band_num):
    '''
    Reads band_num layer of 3D cube of ENVI type to a numpy array.
    Lack of an ENVI .hdr file will cause this to crash.
    '''

    inDs = gdal.Open(file_name, GA_ReadOnly)
    
    if inDs is None:
        print("Couldn't open this file: %s" % file_name)
        print('\nPerhaps you need an ENVI .hdr file?')
        sys.exit("Try again!")
    else:
        print("%s opened successfully" % file_name)
        print('~~~~~~~~~~~~~~')
        print('Get image size')
        print('~~~~~~~~~~~~~~')
        cols = inDs.RasterXSize
        rows = inDs.RasterYSize
        bands = inDs.RasterCount

        print("columns: %i" %cols)
        print("rows: %i" %rows)
        print("bands: %i" %bands)

        print('~~~~~~~~~~~~~~')
        print('Get georeference information')
        print('~~~~~~~~~~~~~~')
        geotransform = inDs.GetGeoTransform()
        originX = geotransform[0]
        originY = geotransform[3]
        pixelWidth = geotransform[1]
        pixelHeight = geotransform[5]

        print("origin x: %i" %originX)
        print("origin y: %i" %originY)
        print("width: %2.2f" %pixelWidth)
        print("height: %2.2f" %pixelHeight)

        # Set pixel offset.....
        print('~~~~~~~~~~~~~~')
        print('Convert image to 2D array')
        print('~~~~~~~~~~~~~~')
        band = inDs.GetRasterBand(band_num + 1)
        image_array = band.ReadAsArray(0, 0, cols, rows)
        image_array_name = file_name
        print("image_array data type: %s" % type(image_array))
        print("image_array dims: %s" % str(image_array.shape))
        
        return image_array, pixelWidth, geotransform, inDs
